# 🚀 YOLOv8 Professional Training (Fine-tuning yolo26n.pt)
**Thiết kế cho Google Colab Pro - Hỗ trợ train 12h+ & Lưu vào Google Drive**

In [ ]:
# 1. Kết nối Google Drive để lưu dữ liệu vĩnh viễn (Phòng trường hợp ngắt kết nối)
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục lưu dự án trên Drive nếu chưa có
import os
save_path = '/content/drive/MyDrive/yolo26n_training'
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f'✅ Đã tạo thư mục lưu trữ: {save_path}')

In [ ]:
# 2. Cài đặt thư viện chuyên dụng
!pip install ultralytics roboflow

In [ ]:
# 3. Tải Dataset Biển Số Xe (24k ảnh)
from roboflow import Roboflow
rf = Roboflow(api_key="maTQLZWb0itL6N8hGL0y")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
dataset = project.version(4).download("yolov8")

In [ ]:
# 4. Cấu hình & Huấn luyện (Fine-tuning)
from ultralytics import YOLO

# ⚠️ BƯỚC QUAN TRỌNG: Upload file yolo26n.pt của bạn lên Colab
# Bạn nên upload vào '/content/yolo26n.pt'
custom_model_path = "/content/yolo26n.pt"

if os.path.exists(custom_model_path):
    model = YOLO(custom_model_path)
    print(f"✅ Đã load model gốc của bạn: {custom_model_path}")
else:
    print("⚠️ Không tìm thấy yolo26n.pt. Đang tải yolov8n.pt để train mới...")
    model = YOLO("yolov8n.pt")

# Bắt đầu train lâu dài
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=500,        # Đặt 500 vòng lặp cho ca train 10-12 tiếng
    imgsz=640,         # Kích thước ảnh tiêu chuẩn
    batch=-1,           # Cài -1 để bộ nhớ tự tối ưu theo GPU của Colab Pro
    patience=100,       # Dừng sớm nếu sau 100 epoch không tiến triển
    project='/content/drive/MyDrive/yolo26n_training', # QUAN TRỌNG: Lưu thẳng vào Drive
    name='run_pro_1',   # Tên phiên bản train
    exist_ok=True,      # Ghi đè nếu trùng tên
    device=0            # Chạy trên GPU
)

In [ ]:
# 5. Sau khi train xong, copy file tốt nhất ra thư mục gốc Drive cho dễ lấy
import shutil
best_model_on_drive = '/content/drive/MyDrive/yolo26n_training/run_pro_1/weights/best.pt'
final_download_path = '/content/drive/MyDrive/yolo26n_updated.pt'

if os.path.exists(best_model_on_drive):
    shutil.copy(best_model_on_drive, final_download_path)
    print(f"🎉 THÀNH CÔNG! Model mới của bạn đã nằm ở: {final_download_path}")
else:
    print("❌ Có lỗi trong quá trình train, không tìm thấy file kết quả.")